In [84]:
import pandas as pd
import json
import requests

In [85]:
response = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/incidentes2019.json")
incidents_db = json.loads(response.text)
response1 = requests.get("https://gitlab.com/drvicsana/cop-proyecto-2023/-/raw/main/project_data/distancias_estaciones_barrios.json")
distances_stations_ntas_db = json.loads(response1.text)
neighbors = distances_stations_ntas_db["42 South Street"].keys()

In [86]:
ev = 197
lv = 143
rv = 5
sv = 8
hv = 1

In [87]:
df =pd.DataFrame()

In [88]:
nta = [elem["nta"] for elem in incidents_db]
units = [elem["units"] for elem in incidents_db]
incident_duration = [elem["incident_duration"] for elem in incidents_db]
first_shift = [elem["is_first_shift"] for elem in incidents_db]
df["nta"]=nta
df["units"]=units
df["incident_duration"]=incident_duration
df["first_shift"]=first_shift

In [89]:
f_shift_db = df[df["first_shift"]==True].reset_index().drop(["index"],axis=1)
s_shift_db = df[df["first_shift"]==False].reset_index().drop(["index"],axis=1)

In [90]:
f_shift_db

,nta,units,incident_duration,first_shift
0,MN03,[ladder],779.0,True
1,MN23,[engine],1935.0,True
2,BK43,[ladder],521.0,True
3,BX14,[engine],1206.0,True
4,BK33,[ladder],730.0,True
...,...,...,...,...
59360,QN26,[ladder],339.0,True
59361,BX44,[engine],710.0,True
59362,MN23,"[engine, ladder, ladder, ladder, engine, engine]",2555.0,True
59363,QN62,[rescue],1860.0,True


In [91]:
units_per_nta_per_shift1 = f_shift_db.groupby(by="nta")["units"].sum()
units_per_nta_per_shift2 = s_shift_db.groupby(by="nta")["units"].sum()

In [92]:
n1 = list(units_per_nta_per_shift1.index)
n2 = list(units_per_nta_per_shift2.index)

In [93]:
ladders = 0
engines = 0
squads = 0
rescues = 0
ha = 0
for elem in units_per_nta_per_shift1:
    
    for e in elem:
        if e == "ladder":ladders+=1
        elif e == "squad": squads+=1
        elif e == "engine": engines+=1
        elif e == "hazardous": ha+=1
        else : rescues+=1

In [94]:
print(ladders)
print(squads)
print(engines)
print(rescues)
print(ha)

56203
2415
47109
4702
2319


In [95]:
ntas = []
engines_vehicles = []
squad_vehicles = []
rescue_vehicles=[]
ha_vehicles = []
ladder_vehicles = []

for i,elem in enumerate(units_per_nta_per_shift1):
    l = elem.count('ladder')
    e = elem.count('engine')
    h = elem.count('hazardous')
    s = elem.count('squad')
    r = elem.count('rescue')
    
    s_l = l/ladders
    s_e = e/engines
    s_h = h/ha
    s_s = s/squads
    s_r = r/rescues

    ntas.append(n1[i])
    engines_vehicles.append(s_e*ev)
    rescue_vehicles.append(s_r*rv)
    squad_vehicles.append(s_s*sv)
    ha_vehicles.append(s_h*hv)
    ladder_vehicles.append(s_l*lv)



    

In [96]:
ladders2 = 0
engines2 = 0
squads2 = 0
rescues2 = 0
ha2 = 0
for elem in units_per_nta_per_shift2:
    
    for e in elem:
        if e == "ladder":ladders2+=1
        elif e == "squad": squads2+=1
        elif e == "engine": engines2+=1
        elif e == "hazardous": ha2+=1
        else : rescues2+=1

In [97]:
for i,elem in enumerate(units_per_nta_per_shift2):
    l = elem.count('ladder')
    e = elem.count('engine')
    h = elem.count('hazardous')
    s = elem.count('squad')
    r = elem.count('rescue')
    
    s_l = l/ladders
    s_e = e/engines
    s_h = h/ha
    s_s = s/squads
    s_r = r/rescues

    ntas.append(n2[i])
    engines_vehicles.append(s_e*ev)
    rescue_vehicles.append(s_r*rv)
    squad_vehicles.append(s_s*sv)
    ha_vehicles.append(s_h*hv)
    ladder_vehicles.append(s_l*lv)




In [98]:
df_final = pd.DataFrame()
df_final["nta"]= ntas
df_final["engines"]=engines_vehicles
df_final["squads"]=squad_vehicles
df_final["ladders"]=ladder_vehicles
df_final["ha"]=ha_vehicles
df_final["rescue"]=rescue_vehicles
df_final["shift"]=[1]*45+[0]*44

In [99]:
df_final

,nta,engines,squads,ladders,ha,rescue,shift
0,BK17,4.934514,0.188820,3.806345,0.028029,0.113781,1
1,BK25,3.671613,0.159006,2.610501,0.018974,0.099957,1
2,BK27,2.396166,0.125880,1.681814,0.011643,0.062739,1
3,BK28,5.043240,0.208696,3.684216,0.028029,0.129732,1
4,BK30,3.378887,0.162319,2.236482,0.016818,0.086134,1
...,...,...,...,...,...,...,...
84,QN52,1.802352,0.023188,1.094070,0.009487,0.005317,0
85,QN55,4.871787,0.039752,2.948900,0.025011,0.057422,0
86,QN62,1.250356,0.013251,0.844724,0.009918,0.011697,0
87,SI07,4.290518,0.046377,2.554526,0.038379,0.034028,0


In [100]:
engines_vehicles

[4.934513574900762,
 3.671612643019381,
 2.3961663376424887,
 5.043240145195186,
 3.378887261457471,
 6.105415101148401,
 3.161434120868624,
 3.479250249421554,
 4.0187013097285025,
 1.7647158717017981,
 5.085058056846887,
 7.999766498970473,
 2.588528731240315,
 8.284128298202042,
 2.6721645545437176,
 4.353244602942113,
 4.9721496953872935,
 6.214141671442824,
 7.543951261966928,
 9.12885011356641,
 2.1118045384109196,
 3.479250249421554,
 6.017597486679827,
 6.117960474643911,
 5.377783438408797,
 8.008130081300813,
 0.19654418476299645,
 3.742703092827273,
 1.8901696066569018,
 4.131609671188096,
 2.42962066696385,
 4.395062514593814,
 2.0741684179243878,
 4.712878643146745,
 1.8692606508310514,
 3.286887855823728,
 3.5127045787429156,
 2.7892547071684817,
 1.881806024326562,
 1.5263537752871001,
 2.3543484259907874,
 6.883228257870046,
 1.710352586554586,
 5.586872996667304,
 14.117726973614385,
 2.5383472372582734,
 2.036532297437857,
 1.6308985544163535,
 1.82326094801418,
 4.21